In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
CWD = 'file_path'
os.chdir(CWD)

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
DF_FILEPATH = 'csv_file'
df = pd.read_csv(DF_FILEPATH)

In [ ]:
df.head()

In [ ]:
import pandas as pd
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Assuming df is your original dataframe with columns ICU stay ID, start time, end time, chart event time

# Convert time columns to datetime
df['starttime'] = pd.to_datetime(df['starttime'])
df['endtime'] = pd.to_datetime(df['endtime'])
df['charttime'] = pd.to_datetime(df['charttime'])

# Create an empty DataFrame to store the hourly intervals
result = pd.DataFrame(columns=['icustay_id', 'time', 'charttimes'])

# Group by ICU stay ID
grouped = df.groupby('icustay_id')

# Iterate through each group
for ICU_stay_ID, group_df in tqdm(grouped, total=len(grouped)):
    interval_start = group_df['starttime'].iloc[0]  # Start time for the ICU stay
    interval_end = group_df['endtime'].iloc[0]  # End time for the ICU stay

    # Generate hourly intervals between start and end times
    hourly_intervals = pd.date_range(start=interval_start, end=interval_end, freq='H')

    # Create a dictionary to store chart events within each hour for this ICU stay ID
    events_within_interval = {interval: [] for interval in hourly_intervals}

    # Store chart events within each hourly interval for this ICU stay ID
    for chart_event_time in group_df['charttime']:
        event_interval = hourly_intervals[(hourly_intervals <= chart_event_time) & (hourly_intervals > chart_event_time - pd.Timedelta(hours=1))]
        if len(event_interval) > 0:
            events_within_interval[event_interval[0]].append(chart_event_time)

    # Create rows for each hourly interval for this ICU stay ID
    for hour_interval, chart_events in events_within_interval.items():
        result = result.append({
            'icustay_id': ICU_stay_ID,
            'time': hour_interval,
            'charttimes': ', '.join(str(event) for event in chart_events)
        }, ignore_index=True)

# result DataFrame now contains hourly intervals with a string of chart events within each interval for each ICU stay ID
print(result)


In [ ]:
df['starttime'] = pd.to_datetime(df['starttime'])
df['endtime'] = pd.to_datetime(df['endtime'])

# Group by icu_stay_id and select the first occurrence of starttime and endtime
unique_df = df.groupby('icustay_id').agg({
    'starttime': 'first',
    'endtime': 'first'
}).reset_index()

In [ ]:
unique_df['icustay_id'].nunique()

In [ ]:
unique_df.head()

In [ ]:
hourly_ranges = pd.DataFrame(columns=['icustay_id', 'start_time', 'end_time'])

for index, row in tqdm(unique_df.iterrows(), total=len(unique_df)):
    stay_id = row['icustay_id']
    start_time = row['starttime']
    end_time = row['endtime']

    # Generate hourly ranges
    intervals = pd.date_range(start=start_time, end=end_time, freq='H')

    # Append ranges to the new DataFrame
    for i in range(len(intervals) - 1):
        hourly_ranges = hourly_ranges.append({
            'icustay_id': stay_id,
            'start_time': intervals[i],
            'end_time': intervals[i + 1]
        }, ignore_index=True)

In [ ]:
115488/72

In [ ]:
df['charttime'] = pd.to_datetime(df['charttime'])
chart_times = []

# Iterate through rows in hourly_ranges and find chart times within each interval
for index, row in tqdm(hourly_ranges.iterrows(), total=len(hourly_ranges)):
    stay_id = row['icustay_id']
    start_time = row['start_time']
    end_time = row['end_time']

    # Filter charttime within the interval for the given icu_stay_id
    charttimes_within_interval = df[(df['icustay_id'] == stay_id) &
                                    (df['charttime'] >= start_time) &
                                    (df['charttime'] < end_time)]['charttime']

    # Format chart times and join into a comma-separated string
    chart_times_str = ', '.join(charttimes_within_interval.dt.strftime('%Y-%m-%d %H:%M:%S %Z'))

    # Append the concatenated chart times to the list
    chart_times.append(chart_times_str)

# Add a new column 'chart_times' to hourly_ranges to store chart times
hourly_ranges['charttimes'] = chart_times
hourly_ranges.to_csv('./ards_filtered_time_interval.csv')
